Librerías y conexión a la base de datos

Se importan las librerías para las bases de datos

In [161]:
import pandas as pd
import numpy as np
#import sqlite3
from openpyxl import Workbook
#from dbi import connect
import pyodbc
#from pyodbc import connect as pyodbc_connect
#from sqlite3 import connect as sqlite3_connect
from scipy import stats
from scipy import special
from scipy.stats import gmean

In [162]:
import funciones as f

In [163]:
anio = int(input('Ingrese el año: '))
mes = int(input('Ingrese el mes: '))

#anio = 2024
#mes = 3

In [164]:
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [165]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [166]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    sql_command = f"EXEC [dbo].[sp_get_calculos_precios_recolectados_mes] {anio}, {mes}"
    cursor.execute(sql_command)
    boletas = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])
    
    sql_command01 = f"EXEC sp_get_indice_grupo {anio}, {mes}"
    cursor.execute(sql_command01)
    ponderaciones = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


In [167]:
#boletas

___________________________________________________________________________________________

In [168]:
participacion = pd.DataFrame({
        'region': [1, 2, 3, 4, 5, 6, 7, 8],
        'participacion': [13.472203, 12.046949, 12.486967, 12.483411, 12.674423, 12.415842, 12.190962, 12.229243]
    })
participacion['region'] = participacion['region'].astype('int64')

In [169]:
bol = boletas[['region', 'codigo_articulo', 'articulo', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo']].copy()
bol.insert(1, 'cod_prod', np.where(bol['codigo_articulo'].str.len() > 8, 
                                   bol['codigo_articulo'].str.slice(0, -2), 
                                   bol['codigo_articulo'].str.slice(0, -1)))
bol.loc[:, 'cod_prod'] = bol['cod_prod'].apply(lambda x: '0' + x if len(x) == 6 else x)
bol['cantidad_anterior'] = pd.to_numeric(bol['cantidad_anterior'], errors='coerce')
bol['cantidad_actual'] = pd.to_numeric(bol['cantidad_actual'], errors='coerce')
bol['precio_anterior'] = pd.to_numeric(bol['precio_anterior'], errors='coerce')
bol['precio_actual'] = pd.to_numeric(bol['precio_actual'], errors='coerce')
bol.loc[bol['cantidad_anterior'] == 0 | bol['cantidad_anterior'].isnull(), 'cantidad_anterior'] = bol['cantidad_actual']
bol.loc[bol['precio_anterior'] == 0 | bol['precio_anterior'].isnull(), 'precio_anterior'] = bol['precio_actual']

#bol['cantidad_actual'] = bol['cantidad_anterior']
#bol['precio_actual'] = bol['precio_anterior']

bol

,region,cod_prod,codigo_articulo,articulo,cantidad_anterior,cantidad_actual,precio_anterior,precio_actual,nt_tipo
0,1,0540301,5403012,Olla y cacerola de 1 a 2lt,1.0,1.0,145.00,145.00,0
1,1,0312306,3123061,Vestidos formales con mangas de largo hasta la...,1.0,1.0,250.00,250.00,0
2,1,0721101,7211011,Llantas para automóvil 185/65 R14,1.0,1.0,375.00,375.00,0
3,1,0411101,4111013,Alquiler de apartamento (con o sin servicios),1.0,1.0,1700.00,1700.00,0
4,1,0321105,3211051,Sandalias para hombre,1.0,1.0,279.99,279.99,0
...,...,...,...,...,...,...,...,...,...
75107,8,0431106,4311061,"Tubo de pvc 1/2""",NaN,NaN,NaN,NaN,0
75108,8,0552101,5521011,"Machete estándar/vizcaíno de (14"" a 24"")",NaN,NaN,NaN,NaN,0
75109,8,0431109,4311091,"Ventanas de pvc blanco de 1x1m, doble corrediza",NaN,NaN,NaN,NaN,0
75110,8,0431107,4311071,Tablas de madera,NaN,NaN,NaN,NaN,0


In [170]:
# Se obtienen las ponderaciones de cada produto por cada region y los indices anteriores
pon_prod = f.pondcat('Producto', ponderaciones).rename(columns={'grupo_codigo':'cod_prod'})
pon_prod['cod_prod'] = pon_prod['cod_prod'].apply(lambda x: '0' + x if len(str(x)) == 6 else x)
pon_prod['ponderacion_region'] = pd.to_numeric(pon_prod['ponderacion_region'], errors='coerce')
pon_prod['indice_anterior'] = pd.to_numeric(pon_prod['indice_anterior'], errors='coerce')
#pon_prod

In [171]:
#pon_prod['ponderacion_region'].dtype
#pon_prod['ponderacion_region'].isnull().sum()

Calculo del IPC

In [172]:
# Se calcula la variacion de las variedades
variacion = f.variacion(bol)
variacion['var_prod'] = bol.apply(lambda row: 1 if row['nt_tipo'] in [351761, 351762] else row['var_prod'], axis=1)
variacion = variacion[['region', 'cod_prod', 'codigo_articulo', 'articulo', 'var_prod']]
#variacion['var_prod'].dtype
#variacion

In [173]:
variacion.to_excel('variacionpy.xlsx')

In [174]:
#relativo de precios de los productos
relativo = (variacion.groupby(['region', 'cod_prod'])
            .agg(relativo = ('var_prod', lambda x: gmean(x)))
            .reset_index())
relativo = pd.merge(relativo, pon_prod[['region', 'cod_prod', 'grupo_nombre', 'ponderacion_region', 'indice_anterior']],
         on=['region', 'cod_prod'], how='left')
relativo
#relativo['ponderacion_region'].dtype

#relativo[relativo['ponderacion_region'] == 0]

,region,cod_prod,relativo,grupo_nombre,ponderacion_region,indice_anterior
0,1,0111101,NaN,Arroz,0.399291,105.375700
1,1,0111102,NaN,Maíz,0.130256,95.976221
2,1,0111201,NaN,Harina de trigo,0.072427,97.930513
3,1,0111202,NaN,Harina de maíz,0.039212,103.073844
4,1,0111203,NaN,Harina para atoles,0.066483,102.158495
...,...,...,...,...,...,...
3185,8,1329101,1.0,Sombrillas y paraguas,0.050085,100.000000
3186,8,1329102,1.0,Ataúdes,0.065264,100.000000
3187,8,1390901,NaN,Servicios legales y financieros,0.194022,100.000000
3188,8,1390902,NaN,Servicios funerarios,0.037005,100.000000


In [175]:
#nulos = relativo['ponderacion_region'].isnull()
#relativo[nulos]
relativo.to_excel('relativopy.xlsx')

In [176]:
#promedio ponderado
prom_pon = relativo
prom_pon['var_ac'] = prom_pon['relativo']*prom_pon['ponderacion_region']
prom_pon = prom_pon[prom_pon['var_ac'] != 0]
## lo unico que se me ocurre es hacer otros dataframes agrupados, uno para la suma de la var ac, otro para la suma de las ponderaciones y asi y luego unirlos

#prom_pon['sum_var_ac'] = prom_pon['var_ac'].sum()
prom_pon01 = prom_pon.groupby(['region', 'cod_prod', 'grupo_nombre']).agg(
    sum_var_ac=('var_ac', 'sum'),
    sum_ponderacion=('ponderacion_region', 'sum')
    ).reset_index()
prom_pon = prom_pon.merge(prom_pon01, on=['region', 'cod_prod', 'grupo_nombre'], how='left')
prom_pon['prom_pon'] = prom_pon['sum_var_ac']/prom_pon['sum_ponderacion']

#prom_pon
#prom_pon01
#prom_pon[prom_pon['var_ac'] == 0]
#prom_pon01[prom_pon01['sum_var_ac'] == 0]

In [177]:
prom_pon.to_excel('promponpy.xlsx')

Índice de precios por región

In [178]:
#índice productos
prom_pon['ind_prod'] = prom_pon['indice_anterior']*prom_pon['prom_pon']
ind_prod = f.cod_grupos(prom_pon).dropna()

ind_prod01 = ind_prod[['region', 'cod_prod', 'ind_prod']].copy()
#dist100['ind_prod'].dtype

In [179]:
ind_prod01.to_excel('indprodpy.xlsx')

In [180]:
#indice subclase
f.indices(ind_prod,'cod_subclase')

,region,grupo_codigo,ind
0,1,03122,98.095820
1,1,03123,100.241754
2,1,03124,99.849332
3,1,03125,94.795362
4,1,03131,101.310814
...,...,...,...
413,8,13120,107.319200
414,8,13131,100.000000
415,8,13132,100.000000
416,8,13291,100.000000


In [181]:
#indice clase
f.indices(ind_prod,'cod_clase')

,region,grupo_codigo,ind
0,1,0312,99.634303
1,1,0313,101.310814
2,1,0321,100.000000
3,1,0431,99.428137
4,1,0521,100.445727
...,...,...,...
308,8,1311,111.904800
309,8,1312,107.319200
310,8,1313,100.000000
311,8,1329,100.000000


In [182]:
#indice grupo
f.indices(ind_prod,'cod_grupo')

,region,grupo_codigo,ind
0,1,031,99.808367
1,1,032,100.000000
2,1,043,99.428137
3,1,052,100.445727
4,1,054,101.996304
...,...,...,...
215,8,105,101.227782
216,8,111,100.000000
217,8,131,104.521671
218,8,132,100.000000


In [183]:
#indice division
f.indices(ind_prod,'cod_div')

,region,grupo_codigo,ind
0,1,03,99.892391
1,1,04,99.428137
2,1,05,100.529028
3,1,06,99.131251
4,1,07,101.127758
...,...,...,...
82,8,08,100.376326
83,8,09,100.206543
84,8,10,100.280204
85,8,11,100.000000


Índice a nivel republica

In [184]:
#indice productos
f.indicerep(ind_prod,'cod_prod')

,cod_prod,ind_prod_rep
0,0112101,97.094877
1,0112506,116.663682
2,0113201,98.941403
3,0113301,100.202246
4,0117405,101.865419
...,...,...
257,1390901,100.000000
258,1390902,100.676929
259,1390903,100.000000
260,1390904,100.000000


In [185]:
#indice subclase
f.indicerep(ind_prod,'cod_subclase')

,cod_subclase,ind_prod_rep
0,01121,97.094877
1,01125,116.663682
2,01132,98.941403
3,01133,100.202246
4,01174,101.865419
...,...,...
100,13131,100.000000
101,13132,100.000000
102,13211,100.201813
103,13291,100.000000


In [186]:
#indice clase
f.indicerep(ind_prod,'cod_clase')

,cod_clase,ind_prod_rep
0,0112,108.520736
1,0113,100.090252
2,0117,103.306645
3,0118,100.000000
4,0119,100.000000
...,...,...
67,1312,100.807797
68,1313,100.000000
69,1321,100.201813
70,1329,100.000000


In [187]:
#indice grupo
f.indicerep(ind_prod,'cod_grupo')

,cod_grupo,ind_prod_rep
0,011,105.033498
1,023,101.492757
2,031,100.515504
3,032,100.771548
4,043,100.713272
5,044,100.000000
6,045,102.418547
7,051,100.673434
8,052,99.973479
9,053,100.506926


In [188]:
#indice division
f.indicerep(ind_prod,'cod_div')

,cod_div,ind_prod_rep
0,01,105.033498
1,02,101.492757
2,03,100.621826
3,04,101.700722
4,05,99.853656
5,06,100.192394
6,07,99.700904
7,08,100.345500
8,09,100.298875
9,10,100.339886


Indice general

In [189]:
#indice producos a nivel nacional
def indicenac(df):
    indice = df.groupby('region').apply(lambda x: pd.Series({
        'sum1': (x['ponderacion_region'] * x['ind_prod']).sum(),
        'sum2': x['ponderacion_region'].sum()
    })).reset_index()

    indice['ind_prod_rep'] = indice['sum1'] / indice['sum2']
    indice = indice[['region', 'ind_prod_rep']]

    return indice



In [190]:
indicenac(ind_prod)

,region,ind_prod_rep
0,1,100.030469
1,2,100.139671
2,3,100.437426
3,4,100.463153
4,5,100.862085
5,6,100.202270
6,7,100.575593
7,8,99.612419


In [191]:
def calculate_replicated_indicator(df):
    sum1 = (df['ponderacion_region'] * df['ind_prod']).sum()
    sum2 = df['ponderacion_region'].sum()

    result = pd.DataFrame({
        'sum1': [sum1],
        'sum2': [sum2]
    })

    result['ind_rep'] = result['sum1'] / result['sum2']

    return result

In [192]:
calculate_replicated_indicator(ind_prod)

,sum1,sum2,ind_rep
0,10391.919211,103.573906,100.333371
